In [ ]:
## geotransform有几种方法，rasterio，gdal等
# This file is for testing the geotransform of the image

In [3]:
from osgeo import gdal

source_img_path = '/scratch2/ziyliu/Inpainting/reprojection/014905040010_01_P001.tif'
mask_path = '/scratch2/ziyliu/Inpainting/reprojection/OcclusionMask-56cm.tif'
output_path = '/scratch2/ziyliu/Inpainting/reprojection/mask_reprojected.tif'

src_img = gdal.Open(source_img_path)
geotransform = src_img.GetGeoTransform()
projection = src_img.GetProjection()
xSize = src_img.RasterXSize
ySize = src_img.RasterYSize

minX = geotransform[0]
maxY = geotransform[3]
maxX = minX + geotransform[1] * xSize
minY = maxY + geotransform[5] * ySize

print('geotransform:\n', geotransform)
print('projection:\n', projection)
print('xSize: ', xSize)
print('ySize: ', ySize)
print('minX: ', minX)
print('maxY: ', maxY)
print('maxX: ', maxX)
print('minY: ', minY)


geotransform:
 (-1146.0012968718295, 0.5044175766791915, 0.0, 1046.9970989015512, 0.0, -0.5044175766791915)
projection:
 PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown_based_on_WGS84_ellipsoid",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Oblique_Stereographic"],PARAMETER["latitude_of_origin",-33.9470042367929],PARAMETER["central_meridian",25.5996312642643],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
xSize:  4507
ySize:  4177
minX:  -1146.0012968718295
maxY:  1046.9970989015512
maxX:  1127.408721221287
minY:  -1059.9551188874318


In [4]:
mask_img = gdal.Open(mask_path)
geotransform2 = mask_img.GetGeoTransform()
projection2 = mask_img.GetProjection()

if projection == projection2:
    print("投影信息相同。")
else:
    print("投影信息不同。")

投影信息相同。


In [9]:
warp_options = gdal.WarpOptions(format='GTiff',
                                outputBounds=[minX, minY, maxX, maxY],
                                xRes=geotransform[1], yRes=geotransform[5],
                                dstSRS=projection,
                                resampleAlg=gdal.GRA_NearestNeighbour,
                                dstNodata=255)  # 使用0填充空缺部分

# 执行重投影和剪切
gdal.Warp(output_path, mask_path, options=warp_options)

print(f"Reprojected mask saved to {output_path}")


Reprojected mask saved to /scratch2/ziyliu/Inpainting/reprojection/mask_reprojected.tif


In [8]:
import os
import glob
import shutil

base_dir = '/scratch2/ziyliu/pro_data/final_data/'
target_dir = '/scratch2/ziyliu/pro_data/final_data_reprojected/'

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

mask_filenames = sorted(list(glob.glob(os.path.join(base_dir, '**', '*mask*.tif'), recursive=True)))
img_filenames = [fname.rsplit('_mask', 1)[0] + '.tif' for fname in mask_filenames]

for i, (img_fname, mask_fname) in enumerate(zip(img_filenames, mask_filenames)):
    print(f"Processing {i + 1}/{len(img_filenames)}: {img_fname}")
    src_img = gdal.Open(img_fname)
    geotransform = src_img.GetGeoTransform()
    projection = src_img.GetProjection()
    xSize = src_img.RasterXSize
    ySize = src_img.RasterYSize

    minX = geotransform[0]
    maxY = geotransform[3]
    maxX = minX + geotransform[1] * xSize
    minY = maxY + geotransform[5] * ySize

    mask_img = gdal.Open(mask_fname)
    geotransform2 = mask_img.GetGeoTransform()
    projection2 = mask_img.GetProjection()

    if projection == projection2:
        print("投影信息相同。")
    else:
        print("投影信息不同。")

    # target_dir
    output_path = os.path.join(target_dir, os.path.basename(mask_fname))
    warp_options = gdal.WarpOptions(format='GTiff',
                                    outputBounds=[minX, minY, maxX, maxY],
                                    xRes=geotransform[1], yRes=geotransform[5],
                                    dstSRS=projection,
                                    resampleAlg=gdal.GRA_NearestNeighbour,
                                    dstNodata=255)  # 使用0填充空缺部分

    # 执行重投影和剪切
    gdal.Warp(output_path, mask_fname, options=warp_options)

    # image
    output_path = os.path.join(target_dir, os.path.basename(img_fname))
    shutil.copy(img_fname, output_path)

Processing 1/20: /scratch2/ziyliu/pro_data/final_data/tile0_014422073010_01_P002_copy000.tif
投影信息相同。
Processing 2/20: /scratch2/ziyliu/pro_data/final_data/tile0_014422073010_01_P002_copy001.tif
投影信息相同。
Processing 3/20: /scratch2/ziyliu/pro_data/final_data/tile0_014422073010_01_P002_copy002.tif
投影信息相同。
Processing 4/20: /scratch2/ziyliu/pro_data/final_data/tile0_014422073010_01_P002_copy003.tif
投影信息相同。
Processing 5/20: /scratch2/ziyliu/pro_data/final_data/tile0_014422073010_01_P002_copy004.tif
投影信息相同。
Processing 6/20: /scratch2/ziyliu/pro_data/final_data/tile1_014422073010_01_P002_copy000.tif
投影信息相同。
Processing 7/20: /scratch2/ziyliu/pro_data/final_data/tile1_014422073010_01_P002_copy001.tif
投影信息相同。
Processing 8/20: /scratch2/ziyliu/pro_data/final_data/tile1_014422073010_01_P002_copy002.tif
投影信息相同。
Processing 9/20: /scratch2/ziyliu/pro_data/final_data/tile1_014422073010_01_P002_copy003.tif
投影信息相同。
Processing 10/20: /scratch2/ziyliu/pro_data/final_data/tile1_014422073010_01_P002_copy004.t

In [6]:
import os
import glob
import shutil
from osgeo import gdal
import random

base_dir = '/scratch2/ziyliu/pro_data/final_data_reprojected/'
target_dir = '/scratch2/ziyliu/pro_data/val/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

mask_filenames = sorted(list(glob.glob(os.path.join(base_dir, '**', '*mask*.tif'), recursive=True)))
img_filenames = [fname.rsplit('_mask', 1)[0] + '.tif' for fname in mask_filenames]

# take bottom 30% rows' left half as validation set, and random crop 35 patches of 256x256 in both image and mask
for i, (img_fname, mask_fname) in enumerate(zip(img_filenames, mask_filenames)):    
    print(f"Processing {i + 1}/{len(img_filenames)}: {img_fname}")
     # random crop 70 patch pairs of image and mask
    src_img = gdal.Open(img_fname)
    geotransform = src_img.GetGeoTransform()
    projection = src_img.GetProjection()
    h, w = src_img.RasterYSize, src_img.RasterXSize

    src_mask = gdal.Open(mask_fname)
    mask_geotransform = src_mask.GetGeoTransform()
    mask_projection = src_mask.GetProjection()
    mask_h, mask_w = src_mask.RasterYSize, src_mask.RasterXSize

    assert h == mask_h and w == mask_w

    num = 0

    while num < 70:
        x_off = random.randint(0, int(w * 0.5) - 256)
        y_off = random.randint(int(h * 0.7), h - 256)

        new_gt = (geotransform[0] + x_off * geotransform[1], geotransform[1], 0, geotransform[3] + y_off * geotransform[5], 0, geotransform[5])
        new_mask_gt = (mask_geotransform[0] + x_off * mask_geotransform[1], mask_geotransform[1], 0, mask_geotransform[3] + y_off * mask_geotransform[5], 0, mask_geotransform[5])

        file_name = os.path.basename(img_fname).split('.')[0]
        output_path = os.path.join(target_dir, f'{file_name}_val_crop00{num}.tif')
        output_mask_path = os.path.join(target_dir, f'{file_name}_val_crop00{num}_mask00{num}.tif')

        out_ds = gdal.Translate(output_path, src_img, format='GTiff', srcWin=[x_off, y_off, 256, 256])
        out_mask_ds = gdal.Translate(output_mask_path, src_mask, format='GTiff', srcWin=[x_off, y_off, 256, 256])

        out_ds.SetGeoTransform(new_gt)
        out_ds.SetProjection(projection)
        out_mask_ds.SetGeoTransform(new_mask_gt)
        out_mask_ds.SetProjection(mask_projection)

        mask_crop = out_mask_ds.ReadAsArray()
        if mask_crop.sum() == 0 or mask_crop.shape[0] != 256 or mask_crop.shape[1] != 256:
            print(f"Empty mask, skip {output_path}")
            out_ds = None
            out_mask_ds = None
            os.remove(output_path)
            os.remove(output_mask_path)
            continue

        out_ds = None
        out_mask_ds = None
        num += 1

Processing 1/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy000.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0014.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0019.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0020.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0033.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0033.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0040.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0059.tif
Empty mask, skip /scratch2/ziyliu/pro_data/val/tile0_014422073010_01_P002_copy000_val_crop0062.tif
Processing 2/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy001.ti

In [8]:
import os
import glob
import shutil
from osgeo import gdal
import random

base_dir = '/scratch2/ziyliu/pro_data/final_data_reprojected/'
target_dir = '/scratch2/ziyliu/pro_data/test/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

mask_filenames = sorted(list(glob.glob(os.path.join(base_dir, '**', '*mask*.tif'), recursive=True)))
img_filenames = [fname.rsplit('_mask', 1)[0] + '.tif' for fname in mask_filenames]

# take bottom 30% rows' right half as test set, and random crop 35 patches of 256x256 in both image and mask
for i, (img_fname, mask_fname) in enumerate(zip(img_filenames, mask_filenames)):
    print(f"Processing {i + 1}/{len(img_filenames)}: {img_fname}")
     # random crop 35 patch pairs of image and mask
    src_img = gdal.Open(img_fname)
    geotransform = src_img.GetGeoTransform()
    projection = src_img.GetProjection()
    h, w = src_img.RasterYSize, src_img.RasterXSize

    src_mask = gdal.Open(mask_fname)
    mask_geotransform = src_mask.GetGeoTransform()
    mask_projection = src_mask.GetProjection()
    mask_h, mask_w = src_mask.RasterYSize, src_mask.RasterXSize

    assert h == mask_h and w == mask_w

    num = 0

    while num < 70:
        x_off = random.randint(int(w * 0.5), w - 256)
        y_off = random.randint(int(h * 0.7), h - 256)

        new_gt = (geotransform[0] + x_off * geotransform[1], geotransform[1], 0, geotransform[3] + y_off * geotransform[5], 0, geotransform[5])
        new_mask_gt = (mask_geotransform[0] + x_off * mask_geotransform[1], mask_geotransform[1], 0, mask_geotransform[3] + y_off * mask_geotransform[5], 0, mask_geotransform[5])
        
        file_name = os.path.basename(img_fname).split('.')[0]
        output_path = os.path.join(target_dir, f'{file_name}_test_crop00{num}.tif')
        output_mask_path = os.path.join(target_dir, f'{file_name}_test_crop00{num}_mask00{num}.tif')

        out_ds = gdal.Translate(output_path, src_img, format='GTiff', srcWin=[x_off, y_off, 256, 256])
        out_mask_ds = gdal.Translate(output_mask_path, src_mask, format='GTiff', srcWin=[x_off, y_off, 256, 256])

        out_ds.SetGeoTransform(new_gt)
        out_ds.SetProjection(projection)
        out_mask_ds.SetGeoTransform(new_mask_gt)
        out_mask_ds.SetProjection(mask_projection)

        mask_crop = out_mask_ds.ReadAsArray()
        if mask_crop.sum() == 0 or mask_crop.shape[0] != 256 or mask_crop.shape[1] != 256:
            print(f"Empty mask, skip {output_path}")
            out_ds = None
            out_mask_ds = None
            os.remove(output_path)
            os.remove(output_mask_path)
            continue

        out_ds = None
        out_mask_ds = None
        num += 1

Processing 1/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy000.tif


Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy000_test_crop0032.tif
Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy000_test_crop0048.tif
Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy000_test_crop0050.tif
Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy000_test_crop0065.tif
Processing 2/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy001.tif
Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy001_test_crop0053.tif
Empty mask, skip /scratch2/ziyliu/pro_data/test/tile0_014422073010_01_P002_copy001_test_crop0061.tif
Processing 3/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy002.tif
Processing 4/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_014422073010_01_P002_copy003.tif
Processing 5/20: /scratch2/ziyliu/pro_data/final_data_reprojected/tile0_0144220